In [50]:
import pandas as pd
import glob
import numpy as np
import re
# Import the tabulate library
from tabulate import tabulate
from uszipcode import SearchEngine
import concurrent.futures

In [51]:
# Load dataset
events = pd.read_csv('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Revised Dataset/2013-2017 Bike Events.csv')

# print
pd.set_option('display.max_columns',None)
events.head()

,Fiscal Year,Event ID,Security Category Name,Public Event Name,Event Created Date,Event Date,Event Goal($),Active Registrations,Inactive Registrations,Total Fees Paid,Total of All Confirmed Gifts($),Total Online Gifts($),Teams,Captains,Average Team Size,Team pages with custom URL,Emails Sent,Personal pages with custom URL,Total Offline Confirmed Gifts($),Self Donors,Total From Participant($),Non-self Donors,Total Not From Participant($),Total Team Gifts($),Total Event Gifts($),Total Offline Unconfirmed Gifts($),Street Address,City,State,Zip Code,Internal Event Name,Previous Event ID
0,2016,27155,AKA Bike Events,Bike MS: Sitka 2016,10/6/2015 10:02,9/1/2016 8:42,0,1,0,0.00,0.00,0.00,0,0,0,0,0,0,0.00,0,0.00,1,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,AKA 2016 Bike MS: Sitka,0
1,2016,27154,AKA Bike Events,Bike MS: Fairbanks 2016 (TEST ONLY),10/5/2015 18:02,3/19/2016 14:39,500000,45,0,0.00,0.00,0.00,6,6,4,0,1,0,0.00,0,0.00,45,0.00,0.00,0.00,0.00,2010 2nd Avenue,Fairbanks,AK,99701.00,AKA 2016 Bike MS: Fairbanks,0
2,2016,26886,AKA Bike Events,Bike MS: Anchorage 2016,8/19/2015 12:07,9/1/2016 13:02,0,7,0,0.00,0.00,0.00,2,2,1,0,17,1,0.00,0,0.00,7,0.00,0.00,0.00,0.00,NaN,Anchorage,AK,99501.00,AKA 2016 Bike MS: Anchorage,0
3,2016,27089,AKA Bike Events,Bike MS: Juno 2016,9/29/2015 23:36,12/31/2016 23:39,0,0,0,0.00,0.00,0.00,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,AKA 2016 Bike MS: Juno,0
4,2015,26357,ALC Bike Events,BB test,5/5/2015 13:06,1/1/2015 12:16,10000,0,0,0.00,0.00,0.00,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0.00,0.00,0.00,NaN,NaN,AL,NaN,BB test,0


In [52]:
dictionary = []
# Iterate through columns

# total rows
total_rows = len(events)

for col in events.columns:
    data_type = events[col].dtype
    length = None
    if data_type == object:
        length = events[col].str.len().max()
    
    length_format = '{:,.0f}'.format(length) if length is not None else None
    
    num_nulls = events[col].isnull().sum()
    num_nulls_format = '{:,}'.format(num_nulls)
    percentage_null = (num_nulls/total_rows) * 100
    percentage_null_format = '{:,.2f}%'.format(percentage_null)
    num_distinct = events[col].nunique()
    num_distinct_format = '{:,}'.format(num_distinct)
    
    # Initialize min and max values
    min_value = None
    max_value = None
    
    # Check if column is numeric
    if np.issubdtype(data_type, np.number):
        min_value = events[col].min()
        max_value = events[col].max()
        
    # Calculate avergae for numeric columns
    average = events[col].mean() if np.issubdtype(data_type, np.number) else None
        
    #Sample values
    sample_values = events[col].dropna().sample(5).tolist()
    
    #Append all info to dictionary list
    dictionary.append([col, data_type, length_format, num_nulls_format, percentage_null_format, num_distinct_format, min_value, max_value, average, sample_values])
    
# Create a df for the dictionary
dictionary_df = pd.DataFrame(dictionary, columns=['Column', 'Data_Type', 'Max_Character_Length', 'Null_Value_Count', 'Null_Percentage','Distinct_Value_Count', 'Min_Value', 'Max_Value','Average','Sample_Values'])

# So that Sample_Values are not truncated
pd.set_option('display.max_colwidth', 300)

# sort = dictionary_df.sort_values(by='Null_Value_Count')
# sort

dictionary_df

,Column,Data_Type,Max_Character_Length,Null_Value_Count,Null_Percentage,Distinct_Value_Count,Min_Value,Max_Value,Average,Sample_Values
0,Fiscal Year,int64,None,0,0.00%,5,2013.00,2017.00,2014.63,"[2014, 2014, 2014, 2015, 2013]"
1,Event ID,int64,None,0,0.00%,700,17966.00,29175.00,24028.87,"[21279, 21764, 28915, 19458, 20363]"
2,Security Category Name,object,15,0,0.00%,53,NaN,NaN,NaN,"[OHA Bike Events, FLS Bike Events, CAS Bike Events, MNM Bike Events, NYR Bike Events]"
3,Public Event Name,object,104,0,0.00%,655,NaN,NaN,NaN,"[Bike MS: FedEx Rockin' Ride to Oxford, Bike MS: New York City 2014, Massachusetts - Bike MS: Ride The Vineyard 2013, Bike MS : Country Challenge 2015, RIR Bike MS 2013]"
4,Event Created Date,object,16,0,0.00%,590,NaN,NaN,NaN,"[12/7/2015 9:46, 11/21/2016 14:49, 7/25/2012 10:24, 1/7/2014 10:27, 11/30/2012 12:04]"
5,Event Date,object,16,102,14.57%,569,NaN,NaN,NaN,"[6/8/2013 10:58, 4/12/2014 10:09, 8/3/2013 10:30, 7/8/2017 12:12, 8/4/2013 16:08]"
6,Event Goal($),int64,None,0,0.00%,225,0.00,20000000.00,722502.62,"[1000, 530000, 300000, 25000, 350000]"
7,Active Registrations,int64,None,0,0.00%,421,0.00,14871.00,677.30,"[1896, 2570, 243, 134, 1186]"
8,Inactive Registrations,int64,None,0,0.00%,111,0.00,1063.00,25.62,"[0, 17, 3, 0, 164]"
9,Total Fees Paid,float64,None,0,0.00%,464,0.00,1584950.00,26357.97,"[1235.0, 0.0, 0.0, 0.0, 0.0]"


In [53]:
events.shape[0]

700

# Zip Code

In [54]:
# Convert 'Zip Code' column to string
events['Zip Code'] = events['Zip Code'].astype(str)

# Now apply .str.contains() with the special character pattern
special_char_values = events[events['Zip Code'].str.contains('[^0-9.]', na=False)]['Zip Code']

# Display the values that contain special characters
special_char_values

0      nan
3      nan
4      nan
5      nan
6      nan
      ... 
668    nan
681    nan
694    nan
695    nan
696    nan
Name: Zip Code, Length: 214, dtype: object

In [55]:
# Convert 'Zip Code' column to string to ensure string operations can be performed
events['Zip Code'] = events['Zip Code'].astype(str)

# Filter rows where 'Zip Code' starts with '0'
zip_starts_with_0 = events[events['Zip Code'].str.startswith('0')]

# Display the filtered DataFrame
zip_starts_with_0

,Fiscal Year,Event ID,Security Category Name,Public Event Name,Event Created Date,Event Date,Event Goal($),Active Registrations,Inactive Registrations,Total Fees Paid,Total of All Confirmed Gifts($),Total Online Gifts($),Teams,Captains,Average Team Size,Team pages with custom URL,Emails Sent,Personal pages with custom URL,Total Offline Confirmed Gifts($),Self Donors,Total From Participant($),Non-self Donors,Total Not From Participant($),Total Team Gifts($),Total Event Gifts($),Total Offline Unconfirmed Gifts($),Street Address,City,State,Zip Code,Internal Event Name,Previous Event ID
619,2016,27922,TXH Bike Events,TXH 2016 Bike MS: San Antonio,2/1/2016 10:33,12/31/2016 10:35,999999,0,0,0.00,0.00,0.00,0,0,0,0,0,0,0.00,0,0.00,0,0.00,0.00,0.00,0.00,[START ADDRESS],[START CITY],TX,0.0,TXH 2016 Bike MS: San Antonio [DO NOT USE],0


In [56]:
events['Zip Code'] = events['Zip Code'].astype(str)

In [57]:
# Remove decimals from zip codes
events['Zip Code'] = events['Zip Code'].apply(lambda x: x.split('.')[0])

In [58]:
events['Zip Code']

0        nan
1      99701
2      99501
3        nan
4        nan
       ...  
695      nan
696      nan
697    53072
698    53072
699    53072
Name: Zip Code, Length: 700, dtype: object

# City

In [59]:
num_nulls = events['City'].isnull().sum()
num_nulls

188

In [60]:
# City imputation

# Initialize the SearchEngine
search = SearchEngine()
# Example function to impute missing state based on ZIP code
def impute_state_from_zip(row):
    if pd.isnull(row['City']) and not pd.isnull(row['Zip Code']):
        # Use the uszipcode library to search by ZIP code
        zipcode_info = search.by_zipcode(row['Zip Code'])
        if zipcode_info:
            # If there's a result, return the state associated with the ZIP code
            return zipcode_info.state
    # Return the original state if it's not missing or if ZIP code info wasn't found
    return row['City']

# Apply the function to impute missing 'Donor State' values
events['City'] = events.apply(impute_state_from_zip, axis=1)


In [61]:
# Remove invalid cities

# Initialize the SearchEngine
search = SearchEngine()

# Validation function to check if the city is correct for the zip code
def is_valid_city(zip_code, city):
    zip_info = search.by_zipcode(zip_code)
    return zip_info and zip_info.major_city and zip_info.major_city.lower() == city.lower()

# Use numpy.where to replace invalid cities with NaN
events['City'] = np.where(events.apply(lambda row: is_valid_city(row['Zip Code'], row['City']), axis=1), events['City'], np.nan)

# State

In [62]:
# State value
max_length_value = events['State'].astype(str).apply(len).idxmax()
max_length = events['State'].astype(str).apply(len).max()
value_with_max_length = events.at[max_length_value, 'State']
value_with_max_length

unique_values = events['State'].unique()
print(unique_values)


[nan 'AK' 'AL' 'MS' 'AR' 'AZ' 'CA' 'NV' 'HI' 'CO' 'WY' 'CT' 'DC' 'MD' 'VA'
 'DE' 'FL' 'GA' 'IL' 'IN' 'KS' 'NE' 'MO' 'KY' 'LA' 'NH' 'RI' 'ME' 'MA'
 'VT' 'MI' 'MN' 'ND' 'SD' 'IA' 'NC' 'NJ' 'NM' 'NY' 'OH' 'OK' 'TX' 'OR'
 'PA' 'TN' 'UT' 'WV' 'ID' 'MT' 'WA' 'WI']


In [63]:
num_nulls = events['State'].isnull().sum()
num_nulls

152

In [64]:
# State imputation

# Initialize the SearchEngine
search = SearchEngine()
# Example function to impute missing state based on ZIP code
def impute_state_from_zip(row):
    if pd.isnull(row['State']) and not pd.isnull(row['Zip Code']):
        # Use the uszipcode library to search by ZIP code
        zipcode_info = search.by_zipcode(row['Zip Code'])
        if zipcode_info:
            # If there's a result, return the state associated with the ZIP code
            return zipcode_info.state
    # Return the original state if it's not missing or if ZIP code info wasn't found
    return row['State']

# Apply the function to impute missing 'Donor State' values
events['State'] = events.apply(impute_state_from_zip, axis=1)

In [65]:
search = SearchEngine()

def validate_state(zip_code, state):
    zip_info = search.by_zipcode(zip_code)
    if zip_info and zip_info.state:
        return zip_info.state.lower() == state.lower()
    return False

# Generate a list of validation results
validation_results = [validate_state(row['Zip Code'], row['State']) for index, row in events.iterrows()]

# If you want to print the results:
for index, result in enumerate(validation_results):
    print(f"Row {index} - Zip: {events.iloc[index]['Zip Code']}, State: {events.iloc[index]['State']} - {'Valid' if result else 'Invalid'}")

# validation_results contains True (valid) or False (invalid) for each row

Row 0 - Zip: nan, State: nan - Invalid
Row 1 - Zip: 99701, State: AK - Valid
Row 2 - Zip: 99501, State: AK - Valid
Row 3 - Zip: nan, State: nan - Invalid
Row 4 - Zip: nan, State: AL - Invalid
Row 5 - Zip: nan, State: nan - Invalid
Row 6 - Zip: nan, State: AL - Invalid
Row 7 - Zip: nan, State: nan - Invalid
Row 8 - Zip: nan, State: AL - Invalid
Row 9 - Zip: 39110, State: MS - Valid
Row 10 - Zip: 35611, State: AL - Valid
Row 11 - Zip: 35611, State: AL - Valid
Row 12 - Zip: 36561, State: AL - Valid
Row 13 - Zip: 39056, State: MS - Valid
Row 14 - Zip: 39056, State: MS - Valid
Row 15 - Zip: 39056, State: MS - Valid
Row 16 - Zip: 35611, State: AL - Valid
Row 17 - Zip: 36561, State: AL - Valid
Row 18 - Zip: 36561, State: AL - Valid
Row 19 - Zip: 35756, State: AL - Valid
Row 20 - Zip: nan, State: nan - Invalid
Row 21 - Zip: 36561, State: AL - Valid
Row 22 - Zip: 39157, State: MS - Valid
Row 23 - Zip: 36561, State: AL - Valid
Row 24 - Zip: 35611, State: AL - Valid
Row 25 - Zip: 72118, State: AR

In [66]:
# Remove invalid state

# Initialize the SearchEngine
search = SearchEngine()

# Validation function to check if the city is correct for the zip code
def is_valid_state(zip_code, state):
    zip_info = search.by_zipcode(zip_code)
    return zip_info and zip_info.state and zip_info.state.lower() == state.lower()

# Use numpy.where to replace invalid cities with NaN
events['State'] = np.where(events.apply(lambda row: is_valid_state(row['Zip Code'], row['State']), axis=1), events['State'], np.nan)

# Event Date

In [67]:
# Event Date
max_length_value = events['Event Date'].astype(str).apply(len).idxmax()
max_length = events['Event Date'].astype(str).apply(len).max()
value_with_max_length = events.at[max_length_value, 'Event Date']
value_with_max_length

'12/31/2016 23:39'

In [68]:
# convert to datetime
# 'errors='coerce'' will handle invalid date formats by converting them to NaT
events['Event Date'] = pd.to_datetime(events['Event Date'].str.split('(').str[0], errors='coerce')

In [69]:
events['Event Date']

0     2016-09-01 08:42:00
1     2016-03-19 14:39:00
2     2016-09-01 13:02:00
3     2016-12-31 23:39:00
4     2015-01-01 12:16:00
              ...        
695   2013-08-03 14:24:00
696   2015-08-01 08:15:00
697   2016-08-06 09:27:00
698   2016-08-06 09:47:00
699   2016-08-06 10:28:00
Name: Event Date, Length: 700, dtype: datetime64[ns]

# Event Goal

In [70]:
# Event Goal
# Define a custom function to convert to float
def convert_to_float(value):
    if value is not None:
        return float(value)
    return None

# Apply the custom function to 'Gift Amount($)' column
events['Event Goal($)'] = events['Event Goal($)'].apply(convert_to_float)
events['Event Goal($)']

0           0.00
1      500000.00
2           0.00
3           0.00
4       10000.00
         ...    
695   1500000.00
696   1500000.00
697   1500000.00
698   1500000.00
699   1500000.00
Name: Event Goal($), Length: 700, dtype: float64

# Event ID

In [71]:
num_nulls = events['Event ID'].isnull().sum()
num_nulls

0

In [72]:
events['Event ID']

0      27155
1      27154
2      26886
3      27089
4      26357
       ...  
695    20385
696    24446
697    26766
698    27038
699    27039
Name: Event ID, Length: 700, dtype: int64

# Drop unnecessary columns

In [73]:
columns_to_keep = ['City', 'State', 'Event Date', 'Event ID', 'Event Goal($)']

events = events[columns_to_keep]
events.columns

Index(['City', 'State', 'Event Date', 'Event ID', 'Event Goal($)'], dtype='object')

# Change column names

In [74]:
events.rename(
    columns={"City": "e_city", "State": "e_state", "Event Date": "e_date","Event ID":"eventID","Event Goal($)":"e_goal_amt"},
    inplace=True,
)

events.columns

Index(['e_city', 'e_state', 'e_date', 'eventID', 'e_goal_amt'], dtype='object')

In [75]:
print(events.dtypes)

e_city                object
e_state               object
e_date        datetime64[ns]
eventID                int64
e_goal_amt           float64
dtype: object


# Tables

In [76]:
# participant
event = events[['eventID','e_city','e_state','e_date','e_goal_amt']].copy()
# Remove duplicates
event_cleaned = events.drop_duplicates(subset=['eventID'])

event_cleaned.to_csv('C:/Users/SANTIAGO/OneDrive - Nova Scotia Community College/4 Winter Semester 2024/DBAS 3018 Data Movement and Integration/Assignment 2/Datasets/Table/event.csv', index=False)